In [4]:
import oasis
con = oasis.getTempCredentials()

print()
print("https://{}:{}".format(con["hostname"], con["port"]))
print("Username: " + con["username"])
print("Password: " + con["password"])
print("Database: " + con["dbName"])


from adbnx_adapter.imdb_arangoDB_networkx_adapter import IMDBArangoDB_Networkx_Adapter
ma = IMDBArangoDB_Networkx_Adapter(conn = con)

Requesting new temp credentials.
Temp database ready to use.

https://5904e8d8a65f.arangodb.cloud:8529
Username: TUTr909ucmlucln5bov134a5n
Password: TUT10xh9aodvvdamwl3233hie
Database: TUT2oumf7xn2kop7ac3wmvxtp


In [9]:
import csv
import json
import requests
import sys
import oasis


from pyArango.connection import *
from pyArango.collection import Collection, Edges, Field
from pyArango.graph import Graph, EdgeDefinition
from pyArango.collection import BulkOperation as BulkOperation

In [10]:
# Connect to the temp database
conn = oasis.connect_pyarango(con)
db = conn[con["dbName"]] 

In [7]:
from pyArango.collection import Collection, Field
from pyArango.graph import Graph, EdgeDefinition


class Users(Collection):
    _fields = {
        "user_id": Field(),
#         "age": Field(),
#         "gender": Field()
    }
    
class Movies(Collection):
    _fields = {
        "movie_id": Field(),
#         "movie_title": Field(),
#         "release_data": Field()
    }

class Ratings(Edges): 
    _fields = {
        #user_id and item_id are encoded by _from, _to 
        "rating": Field(),
#         "timestamp": Field()
    }

class IMDBGraph(Graph) :
    _edgeDefinitions = [EdgeDefinition("Ratings", fromCollections=["Users"], toCollections=["Movies"])]
    _orphanedCollections = []

db.createCollection("Users")
db.createCollection("Movies")
db.createCollection("Ratings")
iMDBGraph = db.createGraph("IMDBGraph")

print("Collection/Graph Setup done.")

Collection/Graph Setup done.


In [8]:
collection = db["Users"]
with BulkOperation(collection, batchSize=100) as col:
    with open('data/users.csv', newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='|')
        #Skip header
        next(reader)
        for row in reader:
            user_id,age,gender,occupation,zip = tuple(row)
            doc = col.createDocument()
            doc["_key"] = user_id
#             doc["age"] = age
#             doc["gender"] = gender
            doc.save()

collection = db["Movies"]
with BulkOperation(collection, batchSize=100) as col:
    with open('data/movies.csv', newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='|')
        #Skip header
        next(reader)
        for row in reader:
            movie_id, movie_title , release_date , video_release_date , url , unknown , action , adventure , animation , childrens , comedy , crime , documentary , drama , fantasy , noir , horror , musical , mystery , romance , scifi , thriller , war , western = tuple(row)
            doc = col.createDocument()
            doc["_key"] = movie_id
#             doc["movie_title"] = movie_title
#             doc["release_date"] = release_date
            doc.save()

collection = db["Ratings"]
with BulkOperation(collection, batchSize=1000) as col:
    with open('data/ratings.csv', newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='|')
        #Skip header
        next(reader)
        for row in reader:
            user_id,movie_id,rating,timestamp = tuple(row)
            doc = col.createDocument()
            doc["_from"] = "Users/"+user_id
            doc["_to"] = "Movies/"+movie_id
            doc["ratings"] = rating
#             doc["timestamp"] = timestamp
            doc.save()
        
print("Import Done")

Import Done


In [6]:
imdb_attributes = { 'vertexCollections': {'Users': {},\
       'Movies': {}},\
                              'edgeCollections' : {'Ratings': {'_from', '_to', 'ratings'}}}

In [7]:
g = ma.create_networkx_graph(graph_name = 'IMDBGraph',  graph_attributes =   imdb_attributes)

In [8]:
g.nodes['Users/2']

{'attr_dict': {'_id': 'Users/2'}, 'bipartite': 0}

In [9]:
g.nodes['Movies/4']

{'attr_dict': {'_id': 'Movies/4'}, 'bipartite': 1}

In [10]:
 m4v = [ t[0] for t in g.in_edges('Movies/4')]

In [11]:
from itertools import combinations
m4vucmb = list(combinations(m4v, 2))

In [12]:
m4v[1]

'Users/19'

In [13]:
import networkx as nx
gp = g.to_undirected()
jcp = nx.jaccard_coefficient(gp, m4vucmb)

In [14]:
gs = nx.DiGraph()
for u, v, p in jcp:
  gs.add_edge(u, v, weight = p)
  #print('(%s, %s) -> %.8f' % (u, v, p))

In [15]:
gs.number_of_edges()

9453

In [16]:
!pip install node2vec

In [17]:
from node2vec import Node2Vec
node2vec = Node2Vec(gs, dimensions=32, walk_length=100, num_walks=30, workers=4)

Computing transition probabilities: 100%|██████████| 138/138 [00:01<00:00, 118.51it/s]


In [18]:
model = node2vec.fit(window=10, min_count=1, batch_words=4)

In [19]:
model.wv.most_similar(m4v[5])

[('Users/115', 0.999237060546875),
 ('Users/189', 0.9991105198860168),
 ('Users/593', 0.9990496635437012),
 ('Users/514', 0.9990367889404297),
 ('Users/450', 0.999014675617218),
 ('Users/659', 0.9989994764328003),
 ('Users/472', 0.9989712238311768),
 ('Users/13', 0.9989547729492188),
 ('Users/650', 0.9989433288574219),
 ('Users/407', 0.9989354610443115)]